In [4]:
from inference_sdk import InferenceHTTPClient
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import requests
import json
import os

# --- 설정 정보 ---
API_URL = "https://serverless.roboflow.com"
API_KEY = "UXMsKtjYlQ1FR7uJ73OI" 
MODEL_ID = "yolov8-trash-detections/6"

# 🚨 수정: 분석할 이미지 파일 목록을 리스트로 정의합니다.
IMAGE_FILE_LIST = ["trash1.jpg", "trash2.jpg"] 

# 1. Inference 클라이언트 초기화
CLIENT = InferenceHTTPClient(
    api_url=API_URL,
    api_key=API_KEY
)

def run_detection_and_visualize(client, model_id, image_path):
    """모델 추론을 실행하고 결과를 출력합니다. (image_path는 단일 문자열입니다)"""
    
    # 🚨 이 부분에서 image_path는 단일 문자열입니다.
    if not os.path.exists(image_path):
        print(f"\n❌ 오류: '{image_path}' 파일을 현재 경로에서 찾을 수 없습니다. 파일 이름을 확인해주세요.")
        return

    print(f"🚀 '{image_path}' 이미지를 '{MODEL_ID}' 모델로 분석 중입니다...")
    
    # 2. 이미지 파일 로드 및 API 호출
    try:
        # 추론 실행
        result = client.infer(
            inference_input=image_path,
            model_id=model_id
        )
        
        # 3. 텍스트 결과 분석 및 출력
        predictions = result['predictions']
        total_count = len(predictions)
        summary = {}

        for p in predictions:
            trash_type = p['class']
            summary[trash_type] = summary.get(trash_type, 0) + 1
            
        print("\n" + "="*30)
        print(f"📊 최종 분석 리포트 ({image_path}): 총 {total_count}개 발견")
        print("="*30)
        
        for k, v in summary.items():
            print(f" • {k}: {v}개")
            
        print("\n✅ 탐지 결과가 JSON 형태로 반환되었습니다.")
        # print(json.dumps(result['predictions'][:2], indent=4)) # 상세 JSON 출력은 생략
        
    except requests.exceptions.HTTPError as err:
        print(f"\n❌ API 호출 중 HTTP 오류 발생: {err}")
    except Exception as e:
        print(f"\n❌ 예측 실행 중 오류 발생: {e}")


# --- 메인 실행 (수정된 부분) ---
for filename in IMAGE_FILE_LIST:
    # 리스트의 각 파일명(문자열)을 함수에 전달합니다.
    run_detection_and_visualize(CLIENT, MODEL_ID, filename)
    print("\n" + "*" * 50 + "\n") # 이미지별 결과를 구분하기 위한 구분선

🚀 'trash1.jpg' 이미지를 'yolov8-trash-detections/6' 모델로 분석 중입니다...

📊 최종 분석 리포트 (trash1.jpg): 총 1개 발견
 • plastic bottle: 1개

✅ 탐지 결과가 JSON 형태로 반환되었습니다.

**************************************************

🚀 'trash2.jpg' 이미지를 'yolov8-trash-detections/6' 모델로 분석 중입니다...

📊 최종 분석 리포트 (trash2.jpg): 총 1개 발견
 • glass bottle: 1개

✅ 탐지 결과가 JSON 형태로 반환되었습니다.

**************************************************

